In [1]:
from astropy.time import Time, TimeDelta
from poliastro.twobody import Orbit
from poliastro.bodies import Earth
from astropy import units as u
import numpy as np
import matplotlib.pyplot  as plt

In [2]:
class SRUnscentedKalmanFilter:
    def __init__(self, dim_x, dim_z, fx, hx, Q, R, alpha=1e-3, beta=2, kappa=0):
        self.dim_x = dim_x
        self.dim_z = dim_z
        self.fx = fx
        self.hx = hx
        self.Q = Q  # process noise
        self.R = R  # measurement noise
        
        self.x = np.zeros(dim_x)
        self.P = np.eye(dim_x)
        
        # UKF parameters
        self.alpha = alpha
        self.beta = beta
        self.kappa = kappa
        self.lambd = self.alpha**2 * (dim_x + self.kappa) - dim_x
        
        self.gamma = np.sqrt(dim_x + self.lambd)
        
        # Weights for mean and covariance
        self.Wm = np.full(2 * dim_x + 1, 1 / (2 * (dim_x + self.lambd)))
        self.Wc = np.full(2 * dim_x + 1, 1 / (2 * (dim_x + self.lambd)))
        self.Wm[0] = self.lambd / (dim_x + self.lambd)
        self.Wc[0] = self.lambd / (dim_x + self.lambd) + (1 - self.alpha**2 + self.beta)
        
    def sigma_points(self):
        Psqrt = np.linalg.cholesky(self.P)
        sigma_points = np.zeros((2 * self.dim_x + 1, self.dim_x))
        sigma_points[0] = self.x
        for i in range(self.dim_x):
            sigma_points[i + 1] = self.x + self.gamma * Psqrt[:, i]
            sigma_points[self.dim_x + i + 1] = self.x - self.gamma * Psqrt[:, i]
        return sigma_points

    def predict(self, dt):
        sigma_points = self.sigma_points()
        x_pred = np.zeros(self.dim_x)
        P_pred_sqrt = np.zeros((self.dim_x, self.dim_x))
        
        for i in range(2 * self.dim_x + 1):
            sigma_points[i] = self.fx(sigma_points[i], dt)
            x_pred += self.Wm[i] * sigma_points[i]
        
        for i in range(2 * self.dim_x + 1):
            y = sigma_points[i] - x_pred
            P_pred_sqrt += self.Wc[i] * np.outer(y, y)
        P_pred_sqrt += self.Q
        
        self.x = x_pred
        self.P = P_pred_sqrt
        self.P = np.linalg.cholesky(self.P + self.Q)  # Convert P_pred_sqrt to square root form
    
    def update(self, z):
        sigma_points = self.sigma_points()
        Z = np.zeros((2 * self.dim_x + 1, self.dim_z))
        z_pred = np.zeros(self.dim_z)
        Pz_sqrt = np.zeros((self.dim_z, self.dim_z))
        Pxz_sqrt = np.zeros((self.dim_x, self.dim_z))
        
        for i in range(2 * self.dim_x + 1):
            Z[i] = self.hx(sigma_points[i])
            z_pred += self.Wm[i] * Z[i]
        
        for i in range(2 * self.dim_x + 1):
            y = Z[i] - z_pred
            Pz_sqrt += self.Wc[i] * np.outer(y, y)
        
        Pz_sqrt += self.R
        
        for i in range(2 * self.dim_x + 1):
            x_diff = sigma_points[i] - self.x
            z_diff = Z[i] - z_pred
            Pxz_sqrt += self.Wc[i] * np.outer(x_diff, z_diff)
        
        Pz = np.linalg.cholesky(Pz_sqrt)  # Convert to square root form
        K = Pxz_sqrt @ np.linalg.inv(Pz @ Pz.T)
        self.x += K @ (z - z_pred)
        self.P -= K @ Pz @ Pz.T @ K.T


In [3]:
# Define the Unscented Kalman Filter
class UnscentedKalmanFilter:
    def __init__(self, dim_x, dim_z, fx, hx, Q, R, alpha=1e-3, beta=2, kappa=0):
        self.dim_x = dim_x
        self.dim_z = dim_z
        self.fx = fx
        self.hx = hx
        self.Q = Q # process noise
        self.R = R # meas noise
        
        self.x = np.zeros(dim_x)
        self.P = np.eye(dim_x)
        
        # UKF parameters
        self.alpha = alpha
        self.beta = beta
        self.kappa = kappa
        self.lambd = self.alpha**2 * (dim_x + self.kappa) - dim_x
        
        self.gamma = np.sqrt(dim_x + self.lambd)
        
        # Weights for mean and covariance
        self.Wm = np.full(2 * dim_x + 1, 1 / (2 * (dim_x + self.lambd)))
        self.Wc = np.full(2 * dim_x + 1, 1 / (2 * (dim_x + self.lambd)))
        self.Wm[0] = self.lambd / (dim_x + self.lambd)
        self.Wc[0] = self.lambd / (dim_x + self.lambd) + (1 - self.alpha**2 + self.beta)
        
    def sigma_points(self):
        Psqrt = np.linalg.cholesky(self.P)
        sigma_points = np.zeros((2 * self.dim_x + 1, self.dim_x))
        sigma_points[0] = self.x
        for i in range(self.dim_x):
            sigma_points[i + 1] = self.x + self.gamma * Psqrt[:, i]
            sigma_points[self.dim_x + i + 1] = self.x - self.gamma * Psqrt[:, i]
        return sigma_points

    def predict(self, dt):
        sigma_points = self.sigma_points()
        x_pred = np.zeros(self.dim_x)
        P_pred = np.zeros((self.dim_x, self.dim_x))
        
        for i in range(2 * self.dim_x + 1):
            sigma_points[i] = self.fx(sigma_points[i], dt)
            x_pred += self.Wm[i] * sigma_points[i]
        
        for i in range(2 * self.dim_x + 1):
            y = sigma_points[i] - x_pred
            P_pred += self.Wc[i] * np.outer(y, y)
        P_pred += self.Q
        
        self.x = x_pred
        self.P = P_pred
        #self.p = np.eye(6)
    
    def update(self, z):
        sigma_points = self.sigma_points()
        Z = np.zeros((2 * self.dim_x + 1, self.dim_z))
        z_pred = np.zeros(self.dim_z)
        Pz = np.zeros((self.dim_z, self.dim_z))
        Pxz = np.zeros((self.dim_x, self.dim_z))
        
        for i in range(2 * self.dim_x + 1):
            Z[i] = self.hx(sigma_points[i])
            z_pred += self.Wm[i] * Z[i]
        
        for i in range(2 * self.dim_x + 1):
            y = Z[i] - z_pred
            Pz += self.Wc[i] * np.outer(y, y)
        
        Pz += self.R
        
        for i in range(2 * self.dim_x + 1):
            x_diff = sigma_points[i] - self.x
            z_diff = Z[i] - z_pred
            Pxz += self.Wc[i] * np.outer(x_diff, z_diff)
        
        K = Pxz @ np.linalg.inv(Pz)
        self.x += K @ (z - z_pred)
        self.P -= K @ Pz @ K.T


In [4]:
# Load measurement data from file
npzfile = np.load('GPS_meas.npz', allow_pickle=True)
noisy_measurement = npzfile['measurements']
t_measurement = npzfile['t_measurements']
# Convert measurement times to Time objects
t_measurement_time = Time(t_measurement)

N = len(t_measurement) # Number of measurements.

# Calculate time intervals (dt) between consecutive measurements
dt_list = (t_measurement_time[1:] - t_measurement_time[:-1]) # TimeDelta object. Shape: (720,)


In [5]:
# gps measurement data
meas_pos = noisy_measurement[:, 0:3] # measured position. Shape: (N, 3)
meas_vel = noisy_measurement[:, 3:6] # measured velocity. Shape: (N, 3)
# add more gaussian noise to the measurement
# meas_pos += np.random.normal(0, 10**4, meas_pos.shape)
# meas_vel += np.random.normal(0, 10**0, meas_vel.shape)

In [6]:
# Define the state transition function
def fx(x, dt):
    orbit = Orbit.from_vectors(Earth, x[:3] * u.km, x[3:] * u.km / u.s)
    orbit = orbit.propagate(TimeDelta(dt * u.s)) # seconds!
    new_pos, new_vel = orbit.r.to(u.km).value, orbit.v.to(u.km / u.s).value
    return np.hstack((new_pos, new_vel))

# Define the measurement function
def hx(x):
    return x

In [7]:
# Initial state vector
position = [3235.64171524, 2693.72565982, -5335.42793567] 
velocity = [-4.87430005, 5.89879341, 0.01977648] 
x = np.hstack((position, velocity)) # initial state vector. Shape: (6,)


In [8]:
# Process & measurement noise
Q = np.eye(6)*10**-6 # Process noise covariance. Shape: (6, 6)
Q[:3,:3] *= 10**3
R = np.eye(6)*10**0 # Measurement noise covariance. Shape: (6, 6)
R[:3,:3] *= 10**3

ukf = UnscentedKalmanFilter(dim_x=6, dim_z=6, fx=fx, hx=hx, Q=Q, R=R, alpha=10**-3, beta=2, kappa=0)
ukf.x = x

srukf = SRUnscentedKalmanFilter(dim_x=6, dim_z=6, fx=fx, hx=hx, Q=Q, R=R, alpha=10**-3, beta=2, kappa=0)
srukf.x = x

In [11]:
# N = 100
ukf_states = [x]
srukf_states = [x]
for i in range(1, N):
    print('Iteration:', i)
    ukf.predict(dt_list[i-1].sec)
    ukf.update(noisy_measurement[i])
    ukf_states.append(ukf.x.copy())
    srukf.predict(dt_list[i-1].sec)
    # srukf.update(noisy_measurement[i])
    # srukf_states.append(srukf.x.copy())

# Plotting the results (3,2) plots, first 3 colrums pos x, y, z, last 3 colrums vel x, y, z
ukf_states = np.array(ukf_states)   
srukf_states = np.array(srukf_states)


Iteration: 1


LinAlgError: Matrix is not positive definite